En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

### Import Libraries

In [1]:
#! pip install -U memory_profiler

In [3]:
import json
import pandas as pd
import pyspark
import zipfile
from pyspark.sql import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
%load_ext memory_profiler

### Converting zip to gz

In [23]:
%%bash
cd ../data
unzip -p ../data/farmers-tweets.json.zip | gzip > farmers-tweets.json.gz

### Reading tweets as pyspark object

Initially the data is a sample of tweets `./data/farmers-tweets.json`, afterwards we'll load the complete json file

In [27]:
file_path = "../data/farmers-tweets.json.gz"

The first test is perform the data loading over a RDD sprk object.
`%memit` magic cell allow us measure memory.

In [25]:
%memit
#Creating SparkSession 
spark = SparkSession.builder.appName('readJson').getOrCreate()
#Read file as pyspark object()   
data = spark.read.json(file_path)
#transformation and renaming columns steps
dfcol = data.withColumn("created_at", data["date"].cast('date'))\
                                .withColumn("user_id", data["user.id"])\
                                .withColumn("username", data["user.username"])

df = dfcol.select(col("created_at"), col("user_id"), col("username")).groupBy("created_at", "username").count()
df.sort(df["count"].desc()).show(10) 

peak memory: 126.81 MiB, increment: 0.03 MiB
+----------+---------------+-----+
|created_at|       username|count|
+----------+---------------+-----+
|2021-02-24|MaanDee08215437|   57|
|2021-02-20|MangalJ23056160|   57|
|2021-02-24| preetysaini321|   43|
|2021-02-15| Rajesh09Sangam|   36|
|2021-02-17|    Gurpreetd86|   32|
|2021-02-17| preetysaini321|   30|
|2021-02-15|    AhluwaliaA2|   27|
|2021-02-15|  akshithepatel|   25|
|2021-02-17|     nonymous_i|   23|
|2021-02-24|    jas04361527|   22|
+----------+---------------+-----+
only showing top 10 rows



### Reading tweets as pandas df

In [36]:
file_path = "../data/farmers-tweets.json.zip"

In [30]:
import json
data = [json.loads(line) for line in open(file_path, 'r')]
#print(data[1000:1100])
#data[0]['user']['id']
df = pd.DataFrame(data)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xcc in position 10: invalid continuation byte

In [37]:
file= "../data/farmers-tweets.json"
zip = zipfile.ZipFile(file_path)
with zipfile.ZipFile(file_path, mode="r") as archive:
    for info in archive.infolist():                
        data = [json.loads(line) for line in open(file, 'r')]
        #print(data[1000:1100])
data.head()


FileNotFoundError: [Errno 2] No such file or directory: 'farmers-tweets.json'

In [8]:
df["created_at"] = pd.to_datetime(df["date"]).dt.strftime('%Y-%m-%d')